In [6]:
import pandas as pd
from decimal import Decimal

from carbon import CarbonSimulatorUI, __version__, __date__
print(f"Carbon Version v{__version__} ({__date__})", )

Carbon Version v2.0-beta3 (12/Dec/2022)


# Carbon Simulation - Demo 5-3

Initialize a fast simulator

It is the result of trading on a single order, which

sells 1B USDC at a price of 1500 USDC per ETH (price range is from 1500 to 1000, but the start price, which each one of the trades above goes by, is 1500)

In [7]:
FastSim = CarbonSimulatorUI(pair="ETH/USDC", verbose=False, matching_method='fast', raiseonerror=True)

# FastSim.debug = True

In [8]:
FastSim.add_strategy('USDC', Decimal(1000000000), Decimal(1500), Decimal(1000), 0, None, None)
FastSim.state()['orders']

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,1.000000e+09,1.000000e+09,USDC,False,1500.0,1000.0,1500.0,USDC per ETH,1
1,1,ETHUSDC,ETH,0.000000e+00,0.000000e+00,ETH,True,None,None,None,USDC per ETH,0


## Route by Source

#### AMM buys USDC

tradeBySourceAmount: input =     10.0 ETH , output =   14999.958711 USDC, rate = 1499.9958711
tradeBySourceAmount: input =    100.0 ETH , output =  149995.871286 USDC, rate = 1499.95871286
tradeBySourceAmount: input =   1000.0 ETH , output = 1499587.230921 USDC, rate = 1499.587230921

In [9]:
FastSim.amm_buys('ETH',Decimal(10), execute=False)
FastSim.amm_buys('ETH',Decimal(100), execute=False)
FastSim.amm_buys('ETH',Decimal(1000), execute=False)
results = FastSim.state()['trades']
results

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,False,None,1.499996e+04,USDC,10.0,ETH,ETHUSDC,0,1,1499.995871,USDC per ETH
0,0,0,A,AMM sells 15000USDC buys 10ETH,True,False,None,1.499996e+04,USDC,10.0,ETH,ETHUSDC,[0],1,1499.995871,USDC per ETH
0,1.0,1,0,route #0,False,False,None,1.499959e+05,USDC,100.0,ETH,ETHUSDC,0,1,1499.958713,USDC per ETH
0,1,1,A,AMM sells 149996USDC buys 100ETH,True,False,None,1.499959e+05,USDC,100.0,ETH,ETHUSDC,[0],1,1499.958713,USDC per ETH
0,2.0,2,0,route #0,False,False,None,1.499587e+06,USDC,1000.0,ETH,ETHUSDC,0,1,1499.587231,USDC per ETH
0,2,2,A,AMM sells 1499587USDC buys 1000ETH,True,False,None,1.499587e+06,USDC,1000.0,ETH,ETHUSDC,[0],1,1499.587231,USDC per ETH


## Route by Target

#### AMM sells ETH

tradeByTargetAmount: input =   1500.0 USDC, output =       1.000001 ETH , rate =    0.000666667
tradeByTargetAmount: input =  15000.0 USDC, output =      10.000028 ETH , rate =    0.000666668
tradeByTargetAmount: input = 150000.0 USDC, output =     100.002753 ETH , rate =    0.000666685


In [10]:
# FastSim = CarbonSimulatorUI(pair="USDC/ETH", verbose=False, matching_method='fast', raiseonerror=True)
#
# FastSim.add_strategy('USDC', 0, None, None, Decimal(1000000000), Decimal(1500), Decimal(1000))

FastSim.amm_sells('USDC',Decimal(1500), execute=False)
FastSim.amm_sells('USDC',Decimal(15000), execute=False)
FastSim.amm_sells('USDC',Decimal(150000), execute=False)
results = FastSim.state()['trades']
results

,uid,id,subid,note,aggr,exec,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0.0,0,0,route #0,False,False,None,1.499996e+04,USDC,10.0,ETH,ETHUSDC,0,1,1499.995871,USDC per ETH
0,0,0,A,AMM sells 15000USDC buys 10ETH,True,False,None,1.499996e+04,USDC,10.0,ETH,ETHUSDC,[0],1,1499.995871,USDC per ETH
0,1.0,1,0,route #0,False,False,None,1.499959e+05,USDC,100.0,ETH,ETHUSDC,0,1,1499.958713,USDC per ETH
0,1,1,A,AMM sells 149996USDC buys 100ETH,True,False,None,1.499959e+05,USDC,100.0,ETH,ETHUSDC,[0],1,1499.958713,USDC per ETH
0,2.0,2,0,route #0,False,False,None,1.499587e+06,USDC,1000.0,ETH,ETHUSDC,0,1,1499.587231,USDC per ETH
0,2,2,A,AMM sells 1499587USDC buys 1000ETH,True,False,None,1.499587e+06,USDC,1000.0,ETH,ETHUSDC,[0],1,1499.587231,USDC per ETH
0,3.0,3,0,route #0,False,False,None,2.000000e+00,USDC,1500.0,ETH,ETHUSDC,0,1,0.001333,USDC per ETH
0,3,3,A,AMM sells 2USDC buys 1500ETH,True,False,None,2.000000e+00,USDC,1500.0,ETH,ETHUSDC,[0],1,0.001333,USDC per ETH
0,4.0,4,0,route #0,False,False,None,1.100003e+01,USDC,15000.0,ETH,ETHUSDC,0,1,0.000733,USDC per ETH
0,4,4,A,AMM sells 11USDC buys 15000ETH,True,False,None,1.100003e+01,USDC,15000.0,ETH,ETHUSDC,[0],1,0.000733,USDC per ETH
